# 01. Let's import necessary libraries

In [1]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
import swatmf
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatmf.__file__

- ## work with the latest version of swatmf

In [2]:
swatmf.__version__

'0.3.1'

In [3]:
from swatmf import swatmf_pst_utils

# 02. write swatmf.con file and initial setup

In [6]:
swatmf_pst_utils.create_swatmf_con?

Signature:
swatmf_pst_utils.create_swatmf_con(
    prj_dir,
    swatmf_model,
    sim_start,
    warmup,
    cal_start,
    cal_end,
    subs=None,
    grids=None,
    riv_parm=None,
    baseflow=None,
    time_step=None,
    pp_included=None,
)
Docstring:
create swatmf.con file containg SWAT-MODFLOW model PEST initial settings

Args:
    prj_dir (`str`): Optimization project working directory
    swatmf_model (`str`): SWAT-MODFLOW working directory
    subs (`list`): reach numbers to be extracted
    grids (`list`): grid numbers to be extracted
    sim_start (`str`): simulation start date e.g. '1/1/2000'
    warmup(`int`): warm-up period
    cal_start (`str`): calibration start date e.g., '1/1/2001'
    cal_end (`str`): calibration end date e.g., '12/31/2005'
    time_step (`str`, optional): model time step. Defaults to None ('day'). e.g., 'day', 'month', 'year'
    riv_parm (`str`, optional): river parameter activation. Defaults to None ('n').
    depth_to_water (`str`, optional): ex

In [7]:
# working directory
prj_dir = "D:/test"
swatmf_model = "D:/spark-brc_gits/swatmf_tutorials/models/middle_bosque_1000/SWAT-MODFLOW"
swat_model = "D:/spark-brc_gits/swatmf_tutorials/models/middle_bosque_1000/SWAT"

In [8]:
# calibration period
sim_start = '1/1/1985'
warmup = 0
cal_start = '1/1/1985'
cal_end = '12/31/1985'
# time step
time_step = 'day'
# locations (what our targets)
subs = [58]
grids = [501]

## 02.01 PEST initial setup

In [9]:
# copy all necessary files (exes) to your working direcotry
swatmf_pst_utils.init_setup(prj_dir, swatmf_model, swat_model)

100%|████████████████████████████████████████████████████████████████████████| 12360/12360 [02:43<00:00, 75.43it/s]

 Creating 'main_opt' folder ... passed



100%|████████████████████████████████████████████████████████████████████████| 12306/12306 [02:42<00:00, 75.66it/s]

 Creating 'backup' folder ... passed
 Creating 'echo' folder ... passed


 Creating 'sufi2.in' folder ... passed
 'Absolute_SWAT_Values.txt' file copied ... passed
 'i64pwtadj1.exe' file copied ... passed
 'pestpp-glm.exe' file copied ... passed
 'pestpp-ies.exe' file copied ... passed
 'pestpp-opt.exe' file copied ... passed
 'pestpp-sen.exe' file copied ... passed
 'SUFI2_LH_sample.exe' file copied ... passed
 'Swat_Edit.exe' file copied ... passed
 'forward_run.py' file copied ... passed


In [10]:
swatmf_pst_utils.create_swatmf_con(
    prj_dir, swatmf_model, sim_start, warmup, cal_start, cal_end, subs=subs, grids=grids)

,names,vals
0,prj_dir,D:/test
1,swatmf_model,D:/spark-brc_gits/swatmf_tutorials/models/middle_bosque_1000/SWAT-MODFLOW
2,sim_start,1/1/1985
3,warm-up,0
4,cal_start,1/1/1985
5,cal_end,12/31/1985
6,subs,[58]
7,grids,[501]
8,riv_parm,n
9,baseflow,n


In [17]:
# check MODFLOW model
mname = "modflow.mfn"
m = flopy.modflow.Modflow.load(mname,model_ws=".")
m.check()


modflow MODEL DATA VALIDATION SUMMARY:
  8 Errors:
    DIS package: thin cells (less than checker threshold of 1.0)
    RIV package: rbot below cell bottom
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    UPW package: zero or negative horizontal hydraulic conductivity values
    UPW package: zero or negative vertical hydraulic conductivity values
    UPW package: negative horizontal anisotropy values
    UPW package: horizontal hydraulic conductivity values below che

# 03. Build template files

## 03.01 MODFLOW pval

In [13]:
os.getcwd()

'D:\\test\\main_opt'

In [18]:
# pval file
pval_file = 'mf_1000.pval'

In [19]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
gw_par

,parnme,parval1,tpl
parnme,,,
hk01,hk01,1.00,~ hk01 ~
hk02,hk02,1.00,~ hk02 ~
sy01,sy01,0.01,~ sy01 ~
sy02,sy02,0.01,~ sy02 ~


## 03.02 SWAT model.in file

In [20]:
# model.in file used
sw_par = swatmf_pst_utils.model_in_to_template_file()
sw_par

,parnme,parval1,tpl
parnme,,,
r__CN2.mgt,r__CN2.mgt,0.001,~ CN2 ~
r__SOL_AWC().sol,r__SOL_AWC().sol,0.001,~ SOL_AWC() ~
v__ESCO.hru,v__ESCO.hru,0.001,~ ESCO ~


# 04. Build instruction files

### Let's do initial run!

In [ ]:
pyemu.os_utils.run(r"SWAT-MODFLOW3")

## 04.01 Streamflow (SWAT)

In [ ]:
# extract daily stream discharge
swatmf_pst_utils.extract_day_stf(subs, sim_start, warmup, cal_start, cal_end)

## 04.02 match it with stf_obd file (SWAT)

In [ ]:
swatmf_pst_utils.stf_obd_to_ins('stf_058.txt', 'rch058',cal_start, cal_end)

## 04.03 Depth to watertable (MODFLOW) 

In [ ]:
swatmf_pst_utils.extract_depth_to_water(grids, sim_start, cal_end)

In [ ]:
mf_obs_grid_ids = pd.read_csv(
                    'modflow.obs',
                    sep=r'\s+',
                    usecols=[3, 4],
                    skiprows=2,
                    header=None
                    )
sim_grids = mf_obs_grid_ids.iloc[:, 0].tolist()

In [ ]:
swatmf_pst_utils.extract_depth_to_water(sim_grids, sim_start, cal_end)

## 04.04 match it with modflow.obd file (MODFLOW)

In [ ]:
swatmf_pst_utils.mf_obd_to_ins('dtw_501.txt', 'g_5699', cal_start, cal_end)

# 5. Create PEST control file

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)

In [ ]:
par = pst.parameter_data
par

## 5.1 Assign parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    else:
        par.iloc[i, 6] = 'swat'
print(par)

## 5.2 Adjust initial parameter values and their ranges

In [ ]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 1  
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 1.000000e-02       
        par.iloc[i, 4] = 1.000000e-04
        par.iloc[i, 5] = 0.6  
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

In [ ]:
# CN2
par.loc['cn2', 'parval1'] = 1.001
par.loc['cn2', 'parlbnd'] = 0.8
par.loc['cn2', 'parubnd'] = 1.2
par.loc['cn2', 'offset'] = -1

# ESCO
par.loc['esco', 'parval1'] = 1.001
par.loc['esco', 'parlbnd'] = 0.5
par.loc['esco', 'parubnd'] = 1.5
par.loc['esco', 'offset'] = -1

# sol_awc()
par.loc['sol_awc()', 'parval1'] = 1.001
par.loc['sol_awc()', 'parlbnd'] = 0.5
par.loc['sol_awc()', 'parubnd'] = 1.5
par.loc['sol_awc()', 'offset'] = -1


## 5.3 Assign parameter group name

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-9]
obd

## 5.4 Provide actual observed values to control file

In [ ]:
os.getcwd()

In [ ]:
# Streamflow
stf_obd = pd.read_csv('stf_day.obd.csv',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd[cal_start: cal_end]
stf_obd

In [ ]:
# watertable
dtw_obd = pd.read_csv('dtw_day.obd.csv',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
dtw_obd = dtw_obd[cal_start: cal_end]
dtw_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += dtw_obd[i].dropna().tolist()
    print(i)
for i in obd_order[1:]:
    tot_obd += stf_obd[i].dropna().tolist()
    print(i)
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 6. Create new control file with settings

We can inspect all control data values using the `pst.control_data.formatted_values` attribute. Values are assigned defaults if not specified. Nice.:

In [ ]:
pst.control_data.formatted_values

In [ ]:
pst.control_data.noptmax = 0 # replace 0 with "zero" and see what happens
pst.model_command = 'python forward_run.py'

### add new PEST++ variables like so:

In [ ]:
# check the dictionary again
pst.pestpp_options

In [ ]:
pst.write('mb_zon.pst', version=2)

You can also read and load the existing pest control file.

In [ ]:
pst_read = pyemu.Pst(os.path.join(swatmf_model,"mb_zon.pst"))

In [ ]:
pst_read.parameter_data